# Online Retail Dataset — Exploration des données

**Objectif** :
Explorer le dataset afin de comprendre sa structure, identifier les problèmes de qualité des données et préparer les étapes de nettoyage et d’analyse.

**Source** :
Online Retail Dataset (Kaggle)
https://www.kaggle.com/datasets/ulrikthygepedersen/online-retail-dataset

## Description du dataset (source Kaggle)

Ce jeu de données contient l’ensemble des transactions réalisées entre le
1er décembre 2010 et le 9 décembre 2011 par une entreprise de e-commerce
basée au Royaume-Uni.

L’entreprise ne dispose pas de magasin physique et vend principalement
des articles cadeaux. Une partie de sa clientèle est composée de grossistes.

Chaque ligne du dataset correspond à une ligne de produit dans une facture
(une même facture peut donc apparaître sur plusieurs lignes).

## Imports et configuration

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


## Chargement des données et aperçu général

In [38]:
# Chargement du dataset
df = pd.read_csv('../data/raw/online_retail.csv')
# Affichage des 10 premières lignes
df.head(10)




,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


## Nombre de lignes et colonnes

In [39]:
print("Lignes :", df.shape[0], "\nColonnes :", df.shape[1])

Lignes : 541909 
Colonnes : 8


Le dataset est volumineux avec plus de 540 000 enregistrements et 8 colonnes. C'est un bon volume pour une analyse commerciale fiable.

## Types de données par colonne

In [40]:
# Types de données par colonne
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Les colonnes numériques (Quantity, UnitPrice, CustomerID) coexistent avec du texte (InvoiceNo, StockCode, Description, Country). On remarque déjà que CustomerID a beaucoup de valeurs nulles (406 829 non-null sur 541 909), ce qui indique des clients anonymes.

## Statistiques descriptives

In [41]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Les quantités sont centrées autour de 9 unités en médiane. Les prix unitaires s'étalent largement (min ~0, max ~21k), suggérant un mix de produits très variés. Le CustomerID médian est 15k, mais avec une grande variabilité.

## Valeurs manquantes par colonne

In [42]:

df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Plus d'un quart des enregistrements (135k) n'ont pas de ClientID. Les descriptions sont presque complètes (1.5k manquantes seulement). Les autres colonnes sont bien remplies.

## Valeurs uniques par colonne

In [43]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

4 000+ produits différents et 38 pays représentés. Les 25 900 numéros de facture suggèrent ~25k transactions, mais on a 540k lignes : chaque facture contient plusieurs articles. Le nombre de dates uniques (23k) indique une couverture de plusieurs années.

## Observations et problèmes détectés

Voici les problèmes détectés dans le dataset :

- **CustomerID** : 135 080 transactions (25 % du dataset) n’ont pas d’identifiant client.  
- **Description** : 1 454 lignes manquent de description du produit (moins de 1 % des données).  
- **Quantités négatives** : certaines transactions ont des quantités négatives, ce qui correspond probablement à des retours.  
- **Prix ou montants négatifs** : quelques transactions ont un prix ou un montant total négatif, ce qui semble être des anomalies.  
- **Format de la date** : `InvoiceDate` est enregistré comme texte et devra être converti en format datetime pour les analyses temporelles.  
- **Répartition par pays** : le dataset contient plusieurs pays, mais la majorité des transactions provient du Royaume-Uni.

Vérification rapide des anomalies :

In [44]:

print(f"Quantités négatives : {(df['Quantity'] < 0).sum()}")
print(f"Prix <= 0 : {(df['UnitPrice'] <= 0).sum()}")
print(f"Top 5 pays : {df['Country'].value_counts().head()}")


Quantités négatives : 10624
Prix <= 0 : 2517
Top 5 pays : Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Name: count, dtype: int64


## Analyse et description détaillée de chaque colonne

- **InvoiceNo** : Il s’agit du numéro unique attribué à chaque facture. Il permet de regrouper toutes les lignes correspondant à une même commande.
- **StockCode** : Ce code identifie chaque produit de façon unique dans le catalogue.
- **Description** : Cette colonne fournit le libellé ou la désignation du produit vendu.
- **Quantity** : On retrouve ici la quantité de produits commandés (ou retournés si la valeur est négative) pour chaque ligne de facture.
- **InvoiceDate** : Cette date correspond au moment où la transaction a été enregistrée. Elle est utile pour toutes les analyses temporelles.
- **UnitPrice** : Il s’agit du prix unitaire du produit, exprimé en livres sterling.
- **CustomerID** : Ce champ correspond à l’identifiant du client, ce qui permet de suivre les achats par client et d’étudier la fidélité.
- **Country** : Cette colonne indique le pays du client, ce qui permet d’analyser la répartition géographique des ventes.

## Exemples de transactions anormales

In [45]:
# Exemple de quantité négative (retours)
df[df['Quantity'] < 0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


In [46]:
# Exemples de prix nuls ou négatifs

df[df['UnitPrice']<=0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
1988,536550,85044,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
2024,536552,20950,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
2025,536553,37461,NaN,3,2010-12-01 14:35:00,0.0,NaN,United Kingdom
2026,536554,84670,NaN,23,2010-12-01 14:35:00,0.0,NaN,United Kingdom
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom


## Période couverte par les données

In [47]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print("La période couverte par les données est de",df['InvoiceDate'].min(),"à",df['InvoiceDate'].max())

La période couverte par les données est de 2010-12-01 08:26:00 à 2011-12-09 12:50:00


La période couverte par les données est de 2010-12-01 au 2011-12-09. 
On dispose donc d’un historique d’un peu plus d’un an, ce qui est suffisant pour observer des tendances saisonnières ou des évolutions sur l’année.

## Différents pays présents

In [ ]:
# Affichage des pays uniques et leur nombre
print("Les pays présents dans les données sont :\n\n", df['Country'].unique(),"\n")
print("Nombre de pays :", df['Country'].nunique())

# Pays par nombre de transactions
print("Pays par nombre de transactions :\n\n", df['Country'].value_counts().head(10))

Les pays présents dans les données sont :

 ['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA'] 

Nombre de pays : 38
Pays par nombre de transactions :
 Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: count, dtype: int64
